In [ ]:
import os

# Script for renaming and moving solution files from tester_results to data

In [ ]:
source_path = "/Users/petr/Documents/Projects/Python/AlgorithmTester/analysis/tester_results/DataAnalysis"
dest_path = "/Users/petr/Documents/Projects/Python/AlgorithmTester/data/DataAnalysis"

for root, dirs, files in os.walk(source_path):
    print(f"{root}")
    dir_name: str = root.split("/")[-1]
    for filename in files:
        split = filename.split("_")
        split = [split[0], split[1], split[4]]
        new_name: str = '_'.join(split)
            
        os.rename(f'{root}/{filename}', f'{dest_path}/{dir_name}/{new_name}')